# Week5 - Logistic Regression Assignment

- Run the below cells.
- Complete all of the numbered questions. You may call any packages that we've used in class.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/classification/loan_status_data/loan_status.csv')

## Data Definitions

<ul>
<li><strong>Loan_ID</strong>: A unique loan ID.</li>
<li><strong>Gender</strong>: Either male or female.</li>
<li><strong>Married</strong>: Weather Married(yes) or Not Marttied(No).</li>
<li><strong>Dependents</strong>: Number of persons depending on the client.</li>
<li><strong>Education</strong>: Applicant Education(Graduate or Undergraduate).</li>
<li><strong>Self_Employed</strong>: Self-employed (Yes/No).</li>
<li><strong>ApplicantIncome</strong>: Applicant income.</li>
<li><strong>CoapplicantIncome</strong>: Co-applicant income.</li>
<li><strong>LoanAmount</strong>: Loan amount in thousands.</li>
<li><strong>Loan_Amount_Term</strong>: Terms of the loan in months.</li>
<li><strong>Credit_History</strong>: Credit history meets guidelines.</li>
<li><strong>Property_Area</strong>: Applicants are living either Urban, Semi-Urban or Rural.</li>
<li><strong>Loan_Status</strong>: Loan approved (Y/N).</li>
</ul>

In [2]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            381 non-null    object 
 1   Gender             376 non-null    object 
 2   Married            381 non-null    object 
 3   Dependents         373 non-null    object 
 4   Education          381 non-null    object 
 5   Self_Employed      360 non-null    object 
 6   ApplicantIncome    381 non-null    int64  
 7   CoapplicantIncome  381 non-null    float64
 8   LoanAmount         381 non-null    float64
 9   Loan_Amount_Term   370 non-null    float64
 10  Credit_History     351 non-null    float64
 11  Property_Area      381 non-null    object 
 12  Loan_Status        381 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 38.8+ KB


In [4]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,381.000000,381.000000,381.000000,370.000000,351.000000
mean,3579.845144,1277.275381,104.986877,340.864865,0.837607
std,1419.813818,2340.818114,28.358464,68.549257,0.369338
min,150.000000,0.000000,9.000000,12.000000,0.000000
25%,2600.000000,0.000000,90.000000,360.000000,1.000000
50%,3333.000000,983.000000,110.000000,360.000000,1.000000
75%,4288.000000,2016.000000,127.000000,360.000000,1.000000
max,9703.000000,33837.000000,150.000000,480.000000,1.000000


# Question 1
- Any data quality issues/concerns?
- Any clean up needed?

Data quality issues/concerns:

There are some missing values in the coloumns of Gender, Dependents, Self_Employed, Loan_Amount_Term, and Credit_History.

Some categorical variables are of type object, which might need encoding for modeling.

We can observe some outliers in numerical variables like ApplicantIncome, CoapplicantIncome, LoanAmount.

# Question 2
- What is the distribution of the target (`Loan_Status`)?  
- Do you have any concerns on class imbalances?

In [5]:
# Distribution of the variables in Loan_Status column

value_counts = df['Loan_Status'].value_counts()
print(value_counts)


Loan_Status
Y    271
N    110
Name: count, dtype: int64


We can observe that both Y and N instances in the loan distrubution are imbalanced. This might impact the the model and training data and skew the results. it is important to understand the problem and resolve by exploring various techniques such as imputing and using machine learning algorithms.

# Question 3
- Create and print a list of the variables that you would treat as numerical and another list for the variables that you would treat as categorical.  
- Explain your choices.

In [6]:
numerical_vars = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
categorical_vars = ['Loan_ID','Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']


In [7]:
print("Numerical Variables :", numerical_vars)
print("Categorical Variables :", categorical_vars)

Numerical Variables : ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
Categorical Variables : ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']


I have choosen treat above variables as numerical and categorical respectively for following reasons:

1) Columns with quantitaive or continious data and measurements can be considered as numerical variables.
2) Columns with qualititative data or categorical data can be considered as categorical variables. 

# Question 4
- Split the data into training and test sets.  
- Use 20% of the data for test and a random state of 120.  

In [8]:
from sklearn.model_selection import train_test_split

X = df.drop('Loan_Status', axis=1)
y = df['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=120)


# Question 5
- Create a pipeline to process the numerical data.  
- Create a pipeline to process the categorical data.  

Verify each pipeline contains the columns you would expect using a fit_transform on the training data, i.e., print the shapes of the fit_transforms for each pipeline.

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [10]:
# Numerical pipeline
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
numerical_pipeline

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [11]:
# Categorical pipeline
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
categorical_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [12]:
X_train_numerical_transformed = numerical_pipeline.fit_transform(X_train[numerical_vars])
X_train_categorical_transformed = categorical_pipeline.fit_transform(X_train[categorical_vars])

print("Numerical pipeline shape:", X_train_numerical_transformed.shape)
print("Categorical pipeline shape:", X_train_categorical_transformed.shape)


Numerical pipeline shape: (304, 5)
Categorical pipeline shape: (304, 319)


# Question 6
- Create a pipeline that combines the pre-processing and implements a logistic regression model.  
- Print the accuracy on the training set and the test set.
- Do you have any concerns of overfitting based on the differences between the two accuracy scores?

In [13]:
#Creating a pipeline with pre-processor and logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [14]:
# Combining  pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_vars),
        ('cat', categorical_pipeline, categorical_vars)
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'Credit_History']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Loan_ID', 'Gender',
                                                   'Married', 'Dependents',
                                                   'Education', 'Self_Employed',
                                                   'Property_Area'])])),
                ('classifier', LogisticRegression())])

In [15]:
# Fit and predicting 
pipeline.fit(X_train, y_train)
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

In [16]:
# Accuracies
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Training Accuracy: 0.875
Test Accuracy: 0.8701298701298701


The test accuracy 0.8701 and the training accuracy 0.875 are relatively close to each other, we can say that no case of overfitting seen, We can also explore other classification algorithm in order to gain the confidence in the score.

# Question 7
What would you recommend as potential next steps for continuing to develop and evaluate a model?

#### - Potential next steps would be be:

- Exploring other classification algorithms.
- Performing feature engineering to improve model performance.
- Additionally, we can also evaluate other metrics like precision, recall, F1 score for a more comprehensive assessment.